<a href="https://colab.research.google.com/github/HoseinNekouei/fine_tuning_with_custom_dataset/blob/main/fine_tuning_imdb_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load Dataset**

In [3]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2025-02-24 18:31:19--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  4.61MB/s    in 20s     

2025-02-24 18:31:40 (3.94 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



# **Load the dataset and divide into training and testing set**

In [4]:
from posixpath import join
from pathlib import Path

def read_imdb_split(split_dir):
  split_dir= Path(split_dir)
  texts=[]
  labels=[]

  for label_dir in ['neg', 'pos']:
    for text_file in (split_dir/label_dir).iterdir():

      texts.append(text_file.read_text())
      labels.append(0 if label_dir== 'neg' else 1)

  return texts, labels

train_texts, train_labels= read_imdb_split('/content/aclImdb/train')
test_texts, test_labels= read_imdb_split('/content/aclImdb/test')

shape=','.join(
    [f'train text length: {len(train_texts)}',
    f'train labels length: {len(train_labels)}',
    f'test text length: {len(test_texts)}',
    f'test labels length: {len(test_labels)}'
    ])

print(shape)

train text length: 25000, train labels length: 25000
test text length: 25000, test labels length: 25000


In [52]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_texts, train_labels, test_size= 0.2, stratify= train_labels)

length= ','. join([
    f'X_train length: {len(X_train)}',
    f'X_val length: {len(X_val)}',
    f'y_train length: {len(y_train)}',
    f'y_val length: {len(y_val)}'
])

print(length)

X_train length: 20000,X_val length: 5000,y_train length: 20000,y_val length: 5000


# **Load Tokenizer**

In [53]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [81]:
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length= 256, return_tensors= 'pt')
val_encodings= tokenizer(X_val, padding= True, truncation= True, max_length= 256, return_tensors= 'pt')
test_encodings= tokenizer(test_texts, padding= True, truncation= True, max_length= 256, return_tensors= 'pt')

# **Make Dataset and DataLoader**

In [55]:
import torch

In [91]:
class IMDBDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings= encodings
    self.labels= labels

  def __getitem__(self, idx):
    item= {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor[self.labels[idx]]
    return item

  def __len__(self):
    return len(self.labels)


train_set= IMDBDataset(train_encodings, y_train)
val_set= IMDBDataset(val_encodings, y_val)
test_set= IMDBDataset(test_encodings, test_labels)